In [1]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.7/613.7 kB 12.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 5.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import optuna
import gc
import requests
import json
from sklearn.preprocessing import SplineTransformer
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error,r2_score,mean_squared_error
from sklearn.model_selection import TimeSeriesSplit,cross_val_score,KFold
from datetime import date
from datetime import datetime
from sklearn.cluster import KMeans
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import os
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 50)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



/usr/local/python/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
!pip install importnb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.5 MB/s eta 0:00:00


In [7]:
# with the new api
from importnb import imports
with imports("ipynb"):
    import modules



In [8]:
class CFG:
    target = 'tüketim'
    train_path = 'data/train.csv'
    test_path = 'data/sample_submission.csv'
    med_path = 'data/med.csv'
    submission_path='data/sample_submission.csv'
    generation_path='data/TR_enerji_uretimi.csv'
    weatherfrom_wwo='data/izmir_weather_data.csv'
    meteostat_feature_columns = ['temp','dwpt','rhum','prcp','wdir','wspd','pres','coco']
    production_features = True
    consumption_features = False
    yuk_tahmin_plani_features = False

### Read Data

In [9]:
train=pd.read_csv(CFG.train_path).rename(columns={'Dağıtılan Enerji (MWh)':'tüketim'})
test=pd.read_csv(CFG.test_path).rename(columns={'Dağıtılan Enerji (MWh)':'tüketim'})
med=pd.read_csv(CFG.med_path)
submission=pd.read_csv(CFG.submission_path)
train = modules.timeseries_features(train.set_index('Tarih'))
test= modules.timeseries_features(test.set_index('Tarih'))
test['tüketim']=test['tüketim'].replace(0,np.nan)
train.index=pd.to_datetime(train.index)

### Train and Test data merged

In [10]:
all_data=pd.concat([train,test],axis=0)
all_data.index=pd.to_datetime(all_data.index)
med.Tarih=pd.to_datetime(med.Tarih)
# Gün içindeki elektrik kesinti süresinin kabul edilebilir limiti aştığı gün.
med['Tarih']=med['Tarih'].astype(str)

## External Data

In [11]:
weather=modules.get_meteostat_data(datetime(2018, 1, 1), datetime(2022, 9, 1),features=CFG.meteostat_feature_columns)[:-1]
generation=pd.read_csv(CFG.generation_path).iloc[:,1:]
generation.Tarih=pd.to_datetime(generation.Tarih)
generation=generation[['total','Tarih']]
# generation=real_time_generation()
consumption=modules.real_time_consumption()
con_pro=consumption.merge(generation,on='Tarih',how='left')
con_pro.set_index('Tarih',drop=True,inplace=True)
all_data=all_data.merge(weather,left_index=True,right_index=True)
all_data=all_data.merge(con_pro,left_index=True,right_index=True)

# ------------------------------------------------------------------------------------------------------------------------------------
yuk=pd.read_csv('data/YukTahmin.csv',encoding='unicode_escape').iloc[:40896,2]
all_data['yuk']=yuk.values
all_data.yuk=all_data.yuk.str.replace(',','').astype(float)
# ------------------------------------------------------------------------------------------------------------------------------------
ep=pd.read_csv('data/DENGESIZLIK.csv',encoding='unicode_escape').iloc[:40896,2:]
all_data['pozitif']=ep['Pozitif Dengesizlik Miktarý (MWh)'].values
all_data['pozitif']=all_data['pozitif'].str.replace(',','').astype(float)
# ------------------------------------------------------------------------------------------------------------------------------------
ue=pd.read_csv('data/UECM.csv',encoding='unicode_escape').iloc[:40896,2:]
ue['UEÇM (MWh)']=ue['UEÇM (MWh)'].str.replace(',','').astype(float)
all_data['ue']=ue['UEÇM (MWh)'].values

# ------------------------------------------------------------------------------------------------------------------------------------
weatherfrom_wwo=pd.read_csv(CFG.weatherfrom_wwo,index_col='date_time')[:'2022-08-31 23:00:00']
weatherfrom_wwo.index=pd.to_datetime(weatherfrom_wwo.index)
all_data=all_data.merge(weatherfrom_wwo[['pressure','humidity','uvIndex','WindGustKmph','windspeedKmph']],left_index=True,right_index=True)
all_index=all_data.index

### Lag Features

In [12]:
all_data.columns

Index(['tüketim', 'hour', 'date', 'dayofweek', 'quarter', 'month', 'year',
       'dayofyear', 'dayofmonth', 'weekday', 'wntr_month', 'month_label',
       'season', 'weekend', 'working_hours', 'label_hour', 'prime_time',
       'temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'pres', 'coco',
       'consumption', 'total', 'yuk', 'pozitif', 'ue', 'pressure', 'humidity',
       'uvIndex', 'WindGustKmph', 'windspeedKmph'],
      dtype='object')

In [13]:
all_data=modules.lag_features(all_data,['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue','pozitif'],48,24) 
# all_data=diff_pct_features(all_data,['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue'],1) 
all_data=modules.rolling_features(all_data,['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue','pozitif'],[6,12,18,24,30,36,42,48],['mean',''])
all_data=modules.rolling_shift_features(all_data,['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue','pozitif'],[6,12,18,24,30,36,42,48],['mean','std'],shift=24)

# hour_df=seasonality_spline_features()
# all_data=all_data.merge(hour_df, on='hour').set_index(all_data.index)

###  Changed some columns of dtypes to category

In [14]:
all_data['med_label']=np.where(all_data.date.isin(med.Tarih),1,0)
all_data=modules.is_categorical(all_data,all_data.columns)

In [15]:
df_train=all_data[all_data.tüketim.notnull()]
df_test=all_data[all_data.tüketim.isnull()]
print(df_train.shape)
print(df_test.shape)

(40152, 296)
(744, 296)


### Target Encoding by some of seasonal columns

In [16]:
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim','hour','mean')
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim',['season','weekend','hour'],'mean')
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim',['season','weekday','prime_time','hour'],'mean')
all_data=pd.concat([df_train,df_test])
all_data.drop(['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue','pozitif'],axis=1,inplace=True)


In [17]:
# all_data=all_data.reset_index(drop=True)
all_data.drop([
'date',
'dayofyear',
'dayofweek',
 # 'weekofyear'
# 'year'
],axis=1,inplace=True)

### Categorical features are labeled for xgboost

In [18]:
cat_features=all_data.select_dtypes('category').columns
le_fit={}
all_data[cat_features],le_fit=modules.label_data(feature=cat_features,data=all_data,le_fit=le_fit)
data_train=all_data[all_data.tüketim.notnull()]
test_sub=all_data[all_data.tüketim.isnull()]
test_sub.drop('tüketim',axis=1,inplace=True)
print(test_sub.shape)
print(data_train.shape)

(744, 285)
(40152, 286)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
test_sub

,hour,quarter,month,year,dayofmonth,weekday,wntr_month,month_label,season,weekend,working_hours,label_hour,prime_time,dwpt,prcp,wdir,wspd,coco,consumption,pressure,windspeedKmph,lag_24_pres,lag_48_pres,lag_24_rhum,lag_48_rhum,...,rolling_shift_24_std_36_ue,rolling_shift_24_mean_42_ue,rolling_shift_24_std_42_ue,rolling_shift_24_mean_48_ue,rolling_shift_24_std_48_ue,rolling_shift_24_mean_6_pozitif,rolling_shift_24_std_6_pozitif,rolling_shift_24_mean_12_pozitif,rolling_shift_24_std_12_pozitif,rolling_shift_24_mean_18_pozitif,rolling_shift_24_std_18_pozitif,rolling_shift_24_mean_24_pozitif,rolling_shift_24_std_24_pozitif,rolling_shift_24_mean_30_pozitif,rolling_shift_24_std_30_pozitif,rolling_shift_24_mean_36_pozitif,rolling_shift_24_std_36_pozitif,rolling_shift_24_mean_42_pozitif,rolling_shift_24_std_42_pozitif,rolling_shift_24_mean_48_pozitif,rolling_shift_24_std_48_pozitif,med_label,hour_te,season_weekend_hour_te,season_weekday_prime_time_hour_te
0,0,2,8,2022,1,0,0,1,3,0,0,2,0,289,0,5,43,1,39107.65,14,15,114,121,54,49,...,4.118585,41.899277,4.164680,41.092238,4.478351,37113.166667,6140.349613,46006.833333,11890.922335,54600.061793,24990.215814,47601.838011,25469.263374,45368.137076,23280.800376,45286.364230,22357.577412,44696.312197,21164.000848,43590.752339,20162.163078,0,1667.289193,1915.604283,1807.355303
1,1,2,8,2022,1,0,0,1,3,0,0,0,0,281,0,356,38,1,37166.97,14,16,111,119,58,52,...,4.145238,41.901995,4.160798,41.052530,4.514116,35499.666667,8286.338677,44491.750000,13540.135135,51055.172904,24604.636441,47527.379678,25539.938763,45025.803742,23531.822072,43646.642007,21705.386144,44323.407435,21401.068590,43184.752339,20370.293209,0,1523.275694,1769.667279,1671.733959
2,2,2,8,2022,1,0,0,1,3,0,0,1,0,271,0,346,30,1,36049.11,14,17,111,118,63,55,...,4.114362,41.742032,4.306678,41.017427,4.557577,33561.500000,10012.400327,42375.333333,14967.037469,47246.228459,23419.565690,47817.671345,25190.874539,44765.003742,23749.310774,43391.753119,21909.822946,43435.097911,21540.875231,43173.460672,20382.168019,0,1420.525514,1657.821245,1561.155129
3,3,2,8,2022,1,0,0,1,3,0,0,0,0,262,0,336,30,1,35100.89,14,18,109,118,69,56,...,4.098883,41.476561,4.455710,40.979346,4.612297,32146.000000,9802.766650,40314.083333,14838.361564,43617.561793,20430.686088,48301.671345,24704.251086,44656.870409,23811.680044,43248.558674,21983.059345,42861.883625,21575.766239,43021.960672,20455.152781,0,1369.097812,1591.400905,1499.191051
4,4,2,8,2022,1,0,0,1,3,0,0,1,0,264,0,338,33,1,34377.08,14,18,110,121,65,55,...,4.130049,41.146617,4.596069,40.928966,4.692386,31529.000000,9017.160662,38833.750000,13791.563430,41597.172904,18732.191123,49006.838011,24188.182454,44713.037076,23798.117746,43249.892007,21982.853333,43110.812197,21478.124601,43003.544006,20457.543113,0,1342.892269,1545.806167,1466.378634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,19,2,8,2022,31,2,0,1,3,1,0,0,0,315,0,315,21,1,44961.13,17,10,155,148,70,56,...,4.681138,38.339567,5.380519,37.991376,5.159010,43522.833333,14259.432786,50340.752990,25133.482083,40426.960039,31065.060183,39453.312846,31679.239288,39742.850277,28646.993147,42148.125231,27677.960575,46273.345436,27960.445587,46603.468923,26537.960996,0,2119.241383,2292.534818,2344.213467
740,20,2,8,2022,31,2,0,1,3,1,0,1,1,318,0,326,15,1,45255.05,17,13,154,151,74,62,...,4.669925,38.567923,5.272747,38.048411,5.174865,36728.166667,14076.135129,44673.836323,23466.029385,41541.345831,29866.222621,38880.354513,31909.241391,38604.916944,28730.899351,41092.819675,27778.369948,45358.393055,28178.883772,46362.135590,26728.880307,0,2101.625539,2284.982010,2335.804114
741,21,2,8,2022,31,2,0,1,3,1,0,0,1,318,0,326,12,1,43948.17,18,8,152,145,76,61,...,4.663447,38.789207,5.121098,38.105539,5.179381,34062.333333,14486.225135,46804.333333,19936.589867,42961.677813,28343.606555,38459.437846,32019.

## Model Tuned

In [20]:
tscv = TimeSeriesSplit(n_splits=3, test_size=2200) 
target=data_train.tüketim
train_labeled=data_train.drop('tüketim',axis=1)
def objective(trial,data=train_labeled,target=target,tscv=tscv):
    
        param = {
            'base_score':0.5,
            'objective':'reg:squarederror',
            'booster':'gbtree', 
            'lambda': trial.suggest_float('lambda', 1e-3, 10),
            'alpha': trial.suggest_float('alpha', 1e-3, 10),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.7,0.8,0.9,1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.5,0.6,0.7,0.8,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate', [0.08,0.009,0.01,0.012,0.014,0.016,0.018,0.001,0.1,0.5]),
            'n_estimators':1500,
            'max_depth': trial.suggest_categorical('max_depth', [3,4,5,7,9,10]),
            'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 500),
            'tree_method':'hist',
            'enable_categorical':True,
            'early_stopping_rounds':250
        } 

        best_iteration_list=[]
        score_list=[]
        model_list=[]
        fea_importance_list=[]
        tscv_splits = list(tscv.split(data,target))  
        for split_train, split_val in tscv_splits:
            x_train, y_train = data.iloc[split_train], target.iloc[split_train]
            x_val, y_val = data.iloc[split_val], target.iloc[split_val]

        model = xgb.XGBRegressor(**param)  
        model.fit(x_train,y_train,eval_set=[(x_train, y_train), (x_val, y_val)],verbose=0)
        best_iteration=model.best_iteration
        val_pred=model.predict(x_val)
        model_list.append(model)
        score=mean_absolute_percentage_error(y_val, val_pred)
        score_list.append(score)
        fea_importance_list.append(model.feature_importances_)


        trial.set_user_attr('model',model_list)
        trial.set_user_attr('best_iteration',best_iteration)
        trial.set_user_attr("feature_importance", np.mean(fea_importance_list,axis=0))
        trial.set_user_attr("score_list", score_list)

        scores=np.mean(score)
        return scores



study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=85)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
gc.collect()

[I 2023-04-30 12:35:27,701] A new study created in memory with name: no-name-675caa1a-9db3-4474-bcf0-47200b0b8cf6
[W 2023-04-30 12:35:29,987] Trial 0 failed with parameters: {'lambda': 5.9627630075645675, 'alpha': 9.324663127256194, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.5, 'max_depth': 5, 'random_state': 24, 'min_child_weight': 312} because of the following error: XGBoostError('[12:35:29] ../src/gbm/gbtree.cc:625: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.\nStack trace:\n  [bt] (0) /usr/local/python/3.10.4/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x2a7e13) [0x7f14137bbe13]\n  [bt] (1) /usr/local/python/3.10.4/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x2ab120) [0x7f14137bf120]\n  [bt] (2) /usr/local/python/3.10.4/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x2ab52a) [0x7f14137bf52a]\n  [bt] (3) /usr/local/python/3.10.4/lib/python3.10/site-packages/xgboost/lib/libxgboost.so

[12:35:29] WARNING: ../src/learner.cc:339: No visible GPU is found, setting `gpu_id` to -1


XGBoostError: [12:35:29] ../src/gbm/gbtree.cc:625: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.
Stack trace:
  [bt] (0) /usr/local/python/3.10.4/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x2a7e13) [0x7f14137bbe13]
  [bt] (1) /usr/local/python/3.10.4/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x2ab120) [0x7f14137bf120]
  [bt] (2) /usr/local/python/3.10.4/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x2ab52a) [0x7f14137bf52a]
  [bt] (3) /usr/local/python/3.10.4/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x2e03c7) [0x7f14137f43c7]
  [bt] (4) /usr/local/python/3.10.4/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x70) [0x7f14136505f0]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.7(+0x6ff5) [0x7f1468c02ff5]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.7(+0x640a) [0x7f1468c0240a]
  [bt] (7) /usr/local/python/3.10.4/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_6

#### BEST PARAMS 
**>params={
'lambda': 5.102807684567904, 
'alpha': 6.511737741485641,
'colsample_bytree': 0.9, 
'subsample': 0.7, 
'learning_rate': 0.012,
'max_depth': 10,
'random_state': 24,
'min_child_weight': 28,
'n_estimators': 1500,
'tree_method':'gpu_hist',
'enable_categorical':True}**


In [ ]:
param={
'lambda': 8.48551751148768, 'alpha': 2.366829085730496, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 0.08, 'max_depth': 5, 'random_state': 48, 'min_child_weight': 24,
    'n_estimators': 1500,
    'tree_method':'gpu_hist',
        'enable_categorical':True
} 

all_models=[]
for i in [1200, 54,55]:
    param["random_state"] = i
    model = xgb.XGBRegressor(**param)  
    model.fit(train_labeled,target)
    all_models.append(model)

preds = [model.predict(test_sub)  for model in all_models]
mean_preds = np.mean(preds, axis=0)


best=pd.read_csv('/kaggle/input/bestresultgediz33/gediz_57.csv')
best['preds']=mean_preds
print(f'8.ay ortalama tüketim :{train[(train.year==2021) &(train.month==8)].tüketim.mean()}')
print(f'8.ay preds ortalama tüketim:{np.mean(mean_preds)}')
importances=model.feature_importances_
features=test_sub.columns
dum=pd.DataFrame(data={'features':features,'importances_rate':importances}).sort_values('importances_rate',ascending=False)
sns.set(rc={'figure.figsize':(30,15)})
g=sns.barplot(x=dum.features,y=dum.importances_rate)
g.set_xticklabels(g.get_xticklabels(), rotation=90);

In [ ]:
submission=pd.read_csv('/kaggle/input/gdz-elektrik-datathon-2023/sample_submission.csv')
submission['Dağıtılan Enerji (MWh)']=mean_preds
submission.set_index('Tarih').to_csv('/kaggle/working/gediz_57.csv')

In [ ]:
plt.plot(best['Dağıtılan Enerji (MWh)'])

In [ ]:
plt.plot(mean_preds)

In [ ]:
best